In [2]:
# %pip install transformers torch datasets evaluate rouge-score sentencepiece accelerate pandas matplotlib

In [3]:
from transformers import (
    pipeline,                          # 고수준 API - 가장 쉬운 방법
    AutoTokenizer,                     # 자동 토크나이저
    AutoModelForSeq2SeqLM,            # 자동 모델 로더 (Seq2Seq 작업용)
    T5TokenizerFast,                  # T5 전용 고속 토크나이저
    T5ForConditionalGeneration,       # T5 모델 클래스
    BartForConditionalGeneration,     # BART 모델 클래스
    PreTrainedTokenizerFast,          # 사전학습 토크나이저 기본 클래스
    DataCollatorForSeq2Seq,           # Seq2Seq 학습용 데이터 콜레이터
    Seq2SeqTrainingArguments,         # Seq2Seq 학습 하이퍼파라미터
    Seq2SeqTrainer,                   # Seq2Seq 전용 트레이너
)

# 경고메세지 숨김
import warnings
warnings.filterwarnings('ignore')
import os
import torch
import numpy as np
from datetime import datetime

In [4]:
# 데이터셋 라이브러리
from datasets import load_dataset

In [5]:
# 버전확인
print(f"pytorch 버전 : {torch.__version__}")
print(f"transformer 버전 : {__import__('transformers').__version__}")

pytorch 버전 : 2.9.1+cpu
transformer 버전 : 4.57.1


### pipeline을 이용한 간단한 문서 요약

In [6]:
# pipeline을 이용한 간단한 문서 요약
'''
모델 다운로드 및 로딩
토크나이제이션(문자-> 숫자)
모델추론(요약생성)
디코딩(숫자-> 문자)

장점: 적으 코드로 실행가능(2~3줄)
단점: 세밀한 제어 어려)

언제:
    빠른 프로토타입 제작
    간단한 데모
    성능테스트
'''
summarizer = pipeline (
    'summarization', #작업유형
    model = "facebook/bart-large-cnn", #facebook 에서 개발한 요약모델(생성형)
    device = 0 if torch.cuda.is_available() else -1

)

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

summary_result = summarizer(ARTICLE,
                            max_length=130, #요약문 최대길이
                            min_length=30, #요약문 최소길이
                            do_sample=False) #매번 동일한 결과(재현성)
print(summary_result[0]['summary_text'])


config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.


In [7]:
summary_text = summary_result[0]['summary_text']
# 통계분석
compression_ratio = len(summary_text) / len(ARTICLE)*100
word_reduced = len(ARTICLE.split()) - len(summary_text.split())
print(f"원문길이 : {len(ARTICLE):4d} 문자 ({len(ARTICLE.split()):3d} 단어)")
print(f"요약문 길이 : {len(summary_text):4d} 문자 ({len(summary_text.split()):3d} 단어)")
print(f"압축비율: {compression_ratio:.2f}%")
print(f"단어감소: {word_reduced} 단어절감")

원문길이 : 2298 문자 (369 단어)
요약문 길이 :  262 문자 ( 47 단어)
압축비율: 11.40%
단어감소: 322 단어절감


### T5 모델과 AutoModel 을 이용한 문서요약

- 모든 nlp작업을 텍스트 -> 텍스트 형식으로 통일
- 요약 : "summarize : [원문] --> [요약문]"
- 번역 : "translate English to French : [원문] --> [번역문]"
- 분류 : "sentiment [원문] -> [클래스]" i love this! --> positive

- 단점 : Task Prefix 필수! (없으면 급격한 성능저하) ---> 위의 예시처럼 문제의 유형을 알려주는 # 짧은 문장을 입력앞에 붙임

- Task Prefix는 T5(T5-small, T5-base, T5-large 등) 모델이 어떤 작업을 수행해야 하는지 알려주는 짧은 “지시문” 입니다
- T5는 Multi-Task 방식으로 학습
  - 요약) summarize: <문장>
  - 번역) translate English to German: <문장>
  - 감정분류) sentiment: <문장>
  - 문법교정) grammar check: <문장>
  - 질문생성) generate question: <문장>

- 모델	파라미터 수	특징
|모델|파라미터수|특징|
|---|------|---|
|t5-small |	약 6억 2천만 |	작고 가벼움, 메모리/속도 빠름, 성능 낮음, 긴 문장/복잡한 task에 약함 |
|t5-base |	약 22억	| t5-small보다 크고 성능 좋음, 긴 문장/복잡한 task 처리 가능, 속도 느림|
|t5-large	| 약 77억	| 더 크고 정확도 높음, 큰 GPU 필요|

In [8]:
#T5 “입력 준비 + 토크나이즈 확인” 과정만 보여주는 코드 (입력전처리 과정을 설명하는 예)

MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# gpu 이동 (가능하면)
device = "cuda" if torch.cuda.is_available() else 'cpu'
model=model.to(device)

# 전처리
# 1. 공백제거
# 2. 줄바꿈을 공백으로 변환(모델은 줄바꿈을 잘 처리 못함)
# 3. Task Prefix 추가 !!필수!!
preprocess_text = ARTICLE.strip().replace("\n", " ")
input_text = f"summarize : {preprocess_text}"  # Task Prefix : summarize

# 토크나이제이션
tokenized_text = tokenizer.encode(
    input_text, return_tensors='pt', truncation=True, max_length=512
).to(device)

print(f"토큰수 : {tokenized_text.shape[1]}")
print(f"텐서형태 : {tokenized_text.shape}, 배치크기=1, 시퀀스길이={tokenized_text.shape[1]}")
print(f"첫 10개토큰 : {tokenized_text[0][:10].tolist()}")

# 디코딩 (숫자->텍스트)
decoded_sample = tokenizer.decode(tokenized_text[0][:20], skip_special_tokens=False)
print(f"첫 20개토큰을 디코딩 : {decoded_sample }")


tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

토큰수 : 512
텐서형태 : torch.Size([1, 512]), 배치크기=1, 시퀀스길이=512
첫 10개토큰 : [21603, 3, 10, 368, 1060, 41, 254, 17235, 61, 10555]
첫 20개토큰을 디코딩 : summarize : New York (CNN)When Liana Barrientos was 23 years old


### BEAM 사용

In [9]:
# 실제 요약생성1. T5 만으로만 요약
# T5+기본 Greedy(단순argmax)
# 장점: 코드 간단, 속도 빠름
# 단점: 매번 가장 가능성 높은 토큰만 선택 → 반복적이거나 단순한 문장 가능
# 길고 복잡한 문장 요약에는 자연스러운 문장 생성이 제한될 수 있음

input_text_t5 = f"summarize: {ARTICLE}"
inputs_t5 = tokenizer(input_text_t5, return_tensors="pt", truncation=True, max_length=512).to(device)
summary_ids_t5 = model.generate(inputs_t5["input_ids"])
summary_t5 = tokenizer.decode(summary_ids_t5[0], skip_special_tokens=True)
print(summary_t5)


# 실제 요약생성2. T5 + Beam Search
# Beam Search 사용 : 여러가능성을 동시에 탐색하면서 최적의 요약을 찾기
# 장점: 여러 후보 시퀀스를 동시에 고려 → 더 자연스럽고 정보가 잘 담긴 요약 가능
# 반복되는 n-gram 방지, 길이 제한, early stopping 등 세밀하게 제어 가능
# 단점: Greedy보다 느림, Beam 수가 크면 메모리/연산 부담 ↑

# beam=4, min_length=30, max_length=100
summary_ids = model.generate(
    tokenized_text,
    num_beams=4,      # Beam Search
    no_repeat_ngram_size=3, #3-gram 반복 방지
    min_length=30,
    max_length=100,
    early_stopping=True  #EOS 만나면 종료
)
# 생성완료
# 디코딩
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f'요약통계')
print(f"원문의 길이 : {len(ARTICLE):4d} 문자")
print(f"요약문의 길이 : {len(output):4d}문자")
print(f"압축률 : {len(output)/len(ARTICLE)*100:.2f}%")
print(f"생성된 토큰수  : {summary_ids.shape[1]}개")
print(f"요약문 : {output}")



# 결론
# 짧고 간단한 요약이면 (T5)Greedy만으로도 충분
# 길고 중요한 정보를 놓치면 안 되는 뉴스 기사, 논문 요약 등은 Beam Search 권장
# Beam 수(num_beams)는 보통 2~5 정도가 적절
# 필요에 따라 no_repeat_ngram_size, min_length, max_length 같은 옵션으로 제어 가능

a year later, she got married again in westchester county, new york. she was
요약통계
원문의 길이 : 2298 문자
요약문의 길이 :  201문자
압축률 : 8.75%
생성된 토큰수  : 51개
요약문 : in 2010, she married once more, this time in the Bronx. she is facing two criminal counts of "offering a false instrument for filing in the first degree" the marriages were part of an immigration scam.


In [ ]:
# T5모델
# Task prefix --> 다양하게 적용가능함
# T5는 Multi-Task 방식으로 학습
# 요약) summarize: <문장>
# 번역) translate English to German: <문장>
# 감정분류) sentiment: <문장>
# 문법교정) grammar check: <문장>
# 질문생성) generate question: <문장>

# 모델	파라미터 수	특징
# t5-small	약 6억 2천만	작고 가벼움, 메모리/속도 빠름, 성능 낮음, 긴 문장/복잡한 task에 약함
# t5-base	약 22억	t5-small보다 크고 성능 좋음, 긴 문장/복잡한 task 처리 가능, 속도 느림
# t5-large	약 77억	더 크고 정확도 높음, 큰 GPU 필요

# T5 Task prefix 참조
https://www.reddit.com/r/MachineLearning/comments/st97z9/d_where_can_i_find_a_list_of_t5_tasksprefixes/

In [2]:
import inspect
from transformers import BertModel

print(inspect.getfile(BertModel))

c:\Users\playdata2\miniconda3\envs\P10\lib\site-packages\transformers\models\bert\modeling_bert.py
